# Hackathon ViladeHack 2022

![](2022-03-26-09-50-38.png)
![](2022-03-26-09-50-09.png)

- Viladecans daily consumption 

	- DATA - Consumption registration date
	- TIPUS_CLIENT - Type of customer (commercial or industrial, in both cases it is non-domestic customers)
	- CONSUM_DIA - Accumulated consumption during the day in m3
	- NUM_CLIENT_ANONIMITZAT - Anonymised customer number
----  
- Monthly consumption Viladecans

	- DATA - Invoice date of consumption
	- NUM_IAE - IAE epigraph of economic activity of the business
	- TEXT_IAE - Descriptive text of the IAE epigraph
	- CONSUM_MES - Accumulated consumption during the two-month billing period (in m3)
	- NUM_CLIENT_ANONIMITZAT - Anonymised customer number

The monthly consumption dataset provides information on the type of economic activity of the customers.  
**The consumption information is not relevant in this dataset.**

## Libraries

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# Import personal library
import Pers_lib as Pers

In [3]:
# settings to display all columns (default is 20, now is None (all))
pd.set_option("display.max_columns", None)

In [4]:
df_month = pd.read_csv('https://raw.githubusercontent.com/viladehack/Viladehack_DataScience/main/Consum_mensual_viladecans_2018_2022_clean.csv',
                       parse_dates=["DATA"], 
                       infer_datetime_format=True)
df_month.head()

,DATA,NUM_IAE,TEXT_IAE,CONSUM_MES,NUM_CLIENT_ANONIMITZAT
0,2018-04-01,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT,14,ODS25IEHI
1,2019-10-01,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT,29,ODS25IEHI
2,2019-12-01,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT,26,ODS25IEHI
3,2021-02-01,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT,0,ODS25IEHI
4,2021-10-01,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT,5,ODS25IEHI


In [5]:
df_day   = pd.read_csv('https://raw.githubusercontent.com/viladehack/Viladehack_DataScience/main/Consum_diari_viladecans_2018_2021.csv',
                       parse_dates=["DATA"], 
                       infer_datetime_format=True)
df_day.head()

,DATA,TIPUS_CLIENT,CONSUM,NUM_CLIENT_ANONIMITZAT
0,2018-01-24,COMERCIAL,0,ODS25IEHI
1,2018-01-03,COMERCIAL,0,ODS25IEHI
2,2018-01-28,COMERCIAL,20,ODS25IEHI
3,2018-01-19,COMERCIAL,31,ODS25IEHI
4,2018-01-18,COMERCIAL,31,ODS25IEHI


In [6]:
Pers.df_info(df_day)

[-------------------------SHAPE------------------------]


(802254, 4)

[-------------------------INFO-------------------------]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802254 entries, 0 to 802253
Data columns (total 4 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   DATA                    802254 non-null  datetime64[ns]
 1   TIPUS_CLIENT            802254 non-null  object        
 2   CONSUM                  802254 non-null  int64         
 3   NUM_CLIENT_ANONIMITZAT  802254 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 24.5+ MB


None

[-----------------------DESCRIBE-----------------------]


...

DATA                      0
TIPUS_CLIENT              0
CONSUM                    0
NUM_CLIENT_ANONIMITZAT    0
dtype: int64

[--------------Values in categorical variables---------]
------------------TIPUS_CLIENT-------------------
------------Unique Values--------------
Number of unique values is: 2
['COMERCIAL' 'INDUSTRIAL']
------------Value Counts--------------


COMERCIAL     726809
INDUSTRIAL     75445
Name: TIPUS_CLIENT, dtype: int64

------------------NUM_CLIENT_ANONIMITZAT-------------------
------------Unique Values--------------
Number of unique values is: 1686
['ODS25IEHI' 'DR0R23EW6' '1BZMYTGZV' ... 'TJJR395X6' '1S8FMJTA5'
 'G02DCTC9T']
------------Value Counts--------------


DBFEA0ILH    1460
X37Q8Y78Y    1460
FDOKN6MBB    1460
NI5UUCKQ7    1459
BBAVWOTEH    1459
             ... 
KH8FZ9JR2       9
3O5CMXQ4J       8
IWV8547XT       8
BKG05GCXO       6
F6N06NPVZ       6
Name: NUM_CLIENT_ANONIMITZAT, Length: 1686, dtype: int64

In [7]:
month =  df_month[["NUM_CLIENT_ANONIMITZAT", "NUM_IAE", "TEXT_IAE" ]]
month.head()

,NUM_CLIENT_ANONIMITZAT,NUM_IAE,TEXT_IAE
0,ODS25IEHI,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT
1,ODS25IEHI,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT
2,ODS25IEHI,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT
3,ODS25IEHI,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT
4,ODS25IEHI,I9429,ALTRES SERVEIS SANITARIS SENSE INTERNAT


In [ ]:
month = month.drop_duplicates()

### CLEANING

Let's first delete the columns we find non-relevant

In [5]:
df_day

,DATA,TIPUS_CLIENT,CONSUM,NUM_CLIENT_ANONIMITZAT
0,2018-01-24,COMERCIAL,0,ODS25IEHI
1,2018-01-03,COMERCIAL,0,ODS25IEHI
2,2018-01-28,COMERCIAL,20,ODS25IEHI
3,2018-01-19,COMERCIAL,31,ODS25IEHI
4,2018-01-18,COMERCIAL,31,ODS25IEHI
...,...,...,...,...
802249,2021-12-12,COMERCIAL,3,G02DCTC9T
802250,2021-12-15,COMERCIAL,-1,G02DCTC9T
802251,2021-12-23,COMERCIAL,50,G02DCTC9T
802252,2021-12-27,COMERCIAL,101,G02DCTC9T


In [6]:
df_month = df_month.drop(columns='Unnamed: 0')

In [7]:
# Group by 'NUM_CLIENT_ANONIMITZAT' the month dataset.
df_month_grouped = df_month.groupby(by=['Code','Class','Business']).count()
df_month_grouped.reset_index()

,Code,Class,Business
0,00VZYC42Y,1,"Comerç al detall de medicaments, productes san..."
1,00VZYC42Y,2,"Comerç al detall de medicaments, productes san..."
2,00VZYC42Y,3,"Comerç al detall de medicaments, productes san..."
3,00VZYC42Y,4,"Comerç al detall de medicaments, productes san..."
4,00VZYC42Y,5,"Comerç al detall de medicaments, productes san..."
...,...,...,...
3414,ZYAFGYRAY,2,"* material i aparells elèctrics, electrodomèsti"
3415,ZYAFGYRAY,3,"* material i aparells elèctrics, electrodomèsti"
3416,ZYAFGYRAY,4,"* material i aparells elèctrics, electrodomèsti"
3417,ZYAFGYRAY,5,"* material i aparells elèctrics, electrodomèsti"


In [10]:
# Join both datasets:
# Let's first  drop columns from df_months that are non-relevant ()
df_merged = df_day.merge(df_month_grouped ,left_on='NUM_CLIENT_ANONIMITZAT' ,right_on='Code' ,how="inner" )
df_merged

,DATA,TIPUS_CLIENT,CONSUM,NUM_CLIENT_ANONIMITZAT
0,2018-01-29,COMERCIAL,45,T53LK6DQK
1,2018-01-29,COMERCIAL,45,T53LK6DQK
2,2018-01-24,COMERCIAL,53,T53LK6DQK
3,2018-01-24,COMERCIAL,53,T53LK6DQK
4,2018-01-03,COMERCIAL,46,T53LK6DQK
...,...,...,...,...
1781618,2021-12-21,COMERCIAL,248,TJJR395X6
1781619,2021-12-21,COMERCIAL,248,TJJR395X6
1781620,2021-12-21,COMERCIAL,248,TJJR395X6
1781621,2021-12-21,COMERCIAL,248,TJJR395X6


In [9]:
list_clients = df_day.NUM_CLIENT_ANONIMITZAT.unique()

df_client_consumption = pd.DataFrame(list_clients)



## PREPROCESS DAILY DATASET
Lets's preprocess the daily dataset